In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from sklearn.cluster import KMeans
from torch.utils.data import DataLoader

# Definice jednoduché konvoluční sítě pro extrakci reprezentací
class FeatureExtractor(nn.Module):
    def __init__(self):
        super(FeatureExtractor, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

    def forward(self, x):
        return self.conv(x)

# Klasifikátor (použijeme po nalezení pseudo-labelů)
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.features = FeatureExtractor()
        self.classifier = nn.Sequential(
            nn.Linear(64*7*7, 128),
            nn.ReLU(),
            nn.Linear(128, 10)  # 10 tříd pro FashionMNIST
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        return self.classifier(x)

# Příprava datasetu a modelu
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# Inicializace modelu pro extrakci reprezentací
feature_extractor = FeatureExtractor()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
feature_extractor.to(device)

# Extrahování latentních reprezentací pro celý dataset
def extract_features(dataloader, model):
    model.eval()  # Přepnutí do eval módu
    features = []
    with torch.no_grad():
        for images, _ in dataloader:
            images = images.to(device)
            outputs = model(images)
            outputs = outputs.view(outputs.size(0), -1)  # Zploštění
            features.append(outputs.cpu())
    return torch.cat(features)

# Extrakce reprezentací z trénovacích dat
features = extract_features(train_loader, feature_extractor)

# Clustering (k-means) na extrahovaných reprezentacích
kmeans = KMeans(n_clusters=10, random_state=0).fit(features)

# Pseudo-labely získané z clusteringu
pseudo_labels = kmeans.labels_

# Vytvoření datasetu s pseudo-labely
class PseudoLabeledDataset(torch.utils.data.Dataset):
    def __init__(self, dataset, pseudo_labels):
        self.dataset = dataset
        self.pseudo_labels = pseudo_labels

    def __getitem__(self, index):
        img, _ = self.dataset[index]
        pseudo_label = self.pseudo_labels[index]
        return img, pseudo_label

    def __len__(self):
        return len(self.dataset)

pseudo_labeled_dataset = PseudoLabeledDataset(train_dataset, pseudo_labels)
pseudo_loader = DataLoader(pseudo_labeled_dataset, batch_size=64, shuffle=True)

# Trénování klasifikátoru na pseudo-labely
classifier = Classifier().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(classifier.parameters(), lr=0.001)

# Trénování klasifikátoru na pseudo-labely
num_epochs = 30

for epoch in range(num_epochs):
    running_loss = 0.0
    for images, labels in pseudo_loader:
        images = images.to(device)
        labels = labels.to(device).long()  # Přidána konverze pseudo-labelů na torch.LongTensor

        # Dopředný průchod
        outputs = classifier(images)
        loss = criterion(outputs, labels)

        # Zpětný průchod a optimalizace
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(pseudo_loader):.4f}')

print('Trénování klasifikátoru s pseudo-labely dokončeno!')

# ------------------------ TESTOVÁNÍ MODELŮ ------------------------

# Testovací data pro vyhodnocení klasifikace
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Funkce pro vyhodnocení přesnosti na testovacích datech
def evaluate_model(dataloader, model):
    model.eval()  # Přepnutí do eval módu (vypnutí dropout/batch norm)
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy

# Vyhodnocení přesnosti na testovací sadě
test_accuracy = evaluate_model(test_loader, classifier)
print(f'Test Accuracy: {test_accuracy:.2f}%')


Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1007)>



100%|██████████| 9912422/9912422 [00:00<00:00, 14504246.28it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1007)>



100%|██████████| 28881/28881 [00:00<00:00, 487491.12it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1007)>



100%|██████████| 1648877/1648877 [00:01<00:00, 1334131.86it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1007)>



100%|██████████| 4542/4542 [00:00<00:00, 3530490.88it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

Epoch [1/30], Loss: 2.2834
Epoch [2/30], Loss: 2.2819
Epoch [3/30], Loss: 2.2818
Epoch [4/30], Loss: 2.2813
Epoch [5/30], Loss: 2.2810
Epoch [6/30], Loss: 2.2805
Epoch [7/30], Loss: 2.2793
Epoch [8/30], Loss: 2.2775
Epoch [9/30], Loss: 2.2749
Epoch [10/30], Loss: 2.2705
Epoch [11/30], Loss: 2.2636
Epoch [12/30], Loss: 2.2550
Epoch [13/30], Loss: 2.2423
Epoch [14/30], Loss: 2.2279
Epoch [15/30], Loss: 2.2100
Epoch [16/30], Loss: 2.1924
Epoch [17/30], Loss: 2.1735
Epoch [18/30], Loss: 2.1556
Epoch [19/30], Loss: 2.1381
Epoch [20/30], Loss: 2.1224
Epoch [21/30], Loss: 2.1064
Epoch [22/30], Loss: 2.0927
Epoch [23/30], Loss: 2.0788
Epoch [24/30], Loss: 2.0645
Epoch [25/30], Loss: 2.0516
Epoch [26/30], Loss: 2.0391
Epoch [27/30], Loss: 2.0280
Epoch [28/30], Loss: 2.0150
Epoch [29/30], Loss: 2.0048
Epoch [30/30], Loss: 1.9931
Trénování klasifikátoru s pseudo-labely dokončeno!
Test Accuracy: 11.01%
